In [1]:
import numpy as np
import cv2
import argparse
import imutils
from scipy.spatial import distance as dist 
import os
import matplotlib.pyplot as plt

In [2]:
# base path to YOLO directory
MODEL_PATH = "yolo-coco"
# initialize minimum probability to filter weak detections along with the threshold when applying non-maxim suppression
MIN_CONF = 0.4
NMS_THRESH = 0.4
# define the minimum safe distance in pixels that two people can be from each other
MIN_DISTANCE = 65

In [3]:
# function to detect people
def detect_people(frame, net, ln, personIdx=0):
    # grab dimensions of the frame and initialize the list of results
    (H, W) = frame.shape[:2]
    results = []

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # initialize lists of detected bounding boxes, centroids, and confidence
    boxes = []
    centroids = []
    confidences = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract teh class ID and confidence of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter detections by (1) ensuring that the object detected was a person and (2) that the minimum confidence is met
            if classID == personIdx and confidence > MIN_CONF:
                # scale the bounding box coordinates back relative to the size of the image
                # YOLO returns the center x and y coordinatesof the bounding box followed by the box width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center x and y coordinates to derive the top and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update the list of bounding box coordinates, centroids and confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes being kept
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # update the results list to consist of the person prediction probability bounding box coordinates, and the centroid
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    return results


In [4]:
# Set the input and output paths
args = {"input": r"C:\Users\maste\Desktop\Image Project\n3.jpg"}

# load the COCO class labels the YOLO model was trained on
labelsPath = os.path.sep.join([MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([MODEL_PATH, "yolov3.cfg"])

# load the YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)


# determine only the "output" layer names that we need from YOLO
ln = net.getLayerNames()
ln = net.getUnconnectedOutLayersNames()

# load the input image
image = cv2.imread(args["input"])

# resize the image for faster processing
image = imutils.resize(image, width=700)
orig = image.copy()

# detect people (only people) in the image
results = detect_people(image, net, ln, personIdx=LABELS.index("person"))

# initialize the set of indexes that violate the minimum social distance
violate = set()

# ensure there are at least two people detections (required in order to compute the the pairwise distance maps)
if len(results) >= 2:
    # extract all centroids from the results and compute the Euclidean distances
    # between all pairs of the centroids
    centroids = np.array([r[2] for r in results])
    D = dist.cdist(centroids, centroids, metric="euclidean")

    # loop over the upper triangular of the distance matrix
    for i in range(0, D.shape[0]):
        for j in range(i+1, D.shape[1]):
            # check to see if the distance between any two centroid pairs is less
            # than the configured number of pixels
            if D[i, j] < MIN_DISTANCE:
                # update the violation set with the indexes of the centroid pairs
                violate.add(i)
                violate.add(j)

# loop over the results
for (i, (prob, bbox, centroid)) in enumerate(results):
    # extract the bounding box and centroid coordinates, then initialize the color of the annotation
    (startX, startY, endX, endY) = bbox
    (cX, cY) = centroid
    color = (0, 255, 0)

    # if the index pair exists within the violation set, then update the color
    if i in violate:
        color = (0, 0, 255)

    # draw (1) a bounding box around the person and (2) the centroid coordinates of the person
    cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
    cv2.circle(image, (cX, cY), 5, color, 1)

# draw the total number of social distancing violations on the output frame
text = "Please Maintain Social Distancing, Violations: {} ".format(len(violate))
cv2.putText(image, text, (10, image.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)

# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)



[INFO] loading YOLO from disk...


-1